# Spark Summit 2019 Demo

In this guide we're going to learn how to run graph algorithms on Apache Spark using the GraphFrames library.

In [49]:
from pyspark.sql.types import *
from graphframes import *
from graphframes.lib import AggregateMessages as AM
from pyspark.sql import functions as F
from pyspark.sql.types import *
from operator import itemgetter

In [45]:
def create_got_graph():
    node_fields = [
        StructField("id", StringType(), True),
        StructField("name", StringType(), True)
    ]
    nodes = spark.read.csv("data/got-s1-nodes.csv", header=True,
                           schema=StructType(node_fields))

    rel_fields = [
        StructField("src", StringType(), True),
        StructField("dst", StringType(), True)
    ]
    rels = spark.read.csv("data/got-s1-edges.csv", header=True,
                         schema=StructType(rel_fields))
    
    reversed_rels = (rels.withColumn("newSrc", rels.dst)
                     .withColumn("newDst", rels.src)
                     .drop("dst", "src")
                     .withColumnRenamed("newSrc", "src")
                     .withColumnRenamed("newDst", "dst"))

    relationships = rels.union(reversed_rels)
    
    return GraphFrame(nodes, relationships)

In [46]:
g = create_got_graph()

In [47]:
g.vertices.show(n=10)

+--------------+---------+
|            id|     name|
+--------------+---------+
|ADDAM_MARBRAND|    Addam|
|         AEGON|    Aegon|
|         AERYS|    Aerys|
|ALLISER_THORNE| Allister|
|          ARYA|     Arya|
|      ASSASSIN| Assassin|
|        BAELOR|   Baelor|
|         BALON|    Balon|
|     BARRISTAN|Barristan|
|        BENJEN|   Benjen|
+--------------+---------+
only showing top 10 rows



In [48]:
g.edges.show(n=10)

+------------+------+
|         src|   dst|
+------------+------+
|         NED|ROBERT|
|    DAENERYS| JORAH|
|         JON|   SAM|
|LITTLEFINGER|   NED|
|         NED| VARYS|
|    DAENERYS| DROGO|
|        ARYA|   NED|
|     CATELYN|  ROBB|
|       BRONN|TYRION|
|      CERSEI|   NED|
+------------+------+
only showing top 10 rows



## PageRank

In [24]:
results = g.pageRank(resetProbability=0.15, tol=0.01)
results.vertices.sort("pagerank", ascending=False).show()

+------------+--------+------------------+
|          id|    name|          pagerank|
+------------+--------+------------------+
|         NED|     Ned| 5.726066795461382|
|      TYRION|  Tyrion|4.1967130251917775|
|     CATELYN| Catelyn| 3.715913757097973|
|      ROBERT|  Robert| 3.437387569020303|
|        ROBB|    Robb| 3.018079866548995|
|        ARYA|    Arya| 2.792500712719572|
|         JON|     Jon|2.6362016956245675|
|      CERSEI|  Cersei|2.6237409076502356|
|     JOFFREY| Joffrey|2.5214197845468864|
|LITTLEFINGER|   Petyr|2.4008909089696475|
|       SANSA|   Sansa|2.3984445662126457|
|       JAIME|   Jaime| 2.281758507933954|
|    DAENERYS|Daenerys|2.1517322572582893|
|        BRAN|    Bran|2.0415657682895043|
|       THEON|   Theon|2.0291290312138353|
|       VARYS|   Varys|1.9941257152972758|
|         PYP|     Pyp|1.9532418374983147|
|       JORAH|   Jorah|1.8996715220003715|
|       DROGO|   Drogo|1.7390308501633556|
|        JEOR|    Jeor|1.7385624576891638|
+----------

## Closeness Centrality

In [50]:
def collect_paths(paths):
    return F.collect_set(paths)


collect_paths_udf = F.udf(collect_paths, ArrayType(StringType()))

paths_type = ArrayType(
    StructType([StructField("id", StringType()), StructField("distance", IntegerType())]))


def flatten(ids):
    flat_list = [item for sublist in ids for item in sublist]
    return list(dict(sorted(flat_list, key=itemgetter(0))).items())


flatten_udf = F.udf(flatten, paths_type)


def new_paths(paths, id):
    paths = [{"id": col1, "distance": col2 + 1} for col1, col2 in paths if col1 != id]
    paths.append({"id": id, "distance": 1})
    return paths


new_paths_udf = F.udf(new_paths, paths_type)


def merge_paths(ids, new_ids, id):
    joined_ids = ids + (new_ids if new_ids else [])
    merged_ids = [(col1, col2) for col1, col2 in joined_ids if col1 != id]
    best_ids = dict(sorted(merged_ids, key=itemgetter(1), reverse=True))
    return [{"id": col1, "distance": col2} for col1, col2 in best_ids.items()]


merge_paths_udf = F.udf(merge_paths, paths_type)


def calculate_closeness(ids):
    nodes = len(ids)
    total_distance = sum([col2 for col1, col2 in ids])
    return 0 if total_distance == 0 else nodes * 1.0 / total_distance


closeness_udf = F.udf(calculate_closeness, DoubleType())

In [51]:
vertices = g.vertices.withColumn("ids", F.array())
cached_vertices = AM.getCachedDataFrame(vertices)
g2 = GraphFrame(cached_vertices, g.edges)

for i in range(0, g2.vertices.count()):
    msg_dst = new_paths_udf(AM.src["ids"], AM.src["id"])
    msg_src = new_paths_udf(AM.dst["ids"], AM.dst["id"])
    agg = g2.aggregateMessages(F.collect_set(AM.msg).alias("agg"),
                               sendToSrc=msg_src, sendToDst=msg_dst)
    res = agg.withColumn("newIds", flatten_udf("agg")).drop("agg")
    new_vertices = (g2.vertices.join(res, on="id", how="left_outer")
                    .withColumn("mergedIds", merge_paths_udf("ids", "newIds", "id"))
                    .drop("ids", "newIds")
                    .withColumnRenamed("mergedIds", "ids"))
    cached_new_vertices = AM.getCachedDataFrame(new_vertices)
    g2 = GraphFrame(cached_new_vertices, g2.edges)

(g2.vertices
 .withColumn("closeness", closeness_udf("ids"))
 .sort("closeness", ascending=False)
 .show(truncate=False))

Py4JJavaError: An error occurred while calling o7545.getCachedDataFrame.
: java.lang.StackOverflowError
	at java.io.ObjectOutputStream$BlockDataOutputStream.drain(ObjectOutputStream.java:1877)
	at java.io.ObjectOutputStream$BlockDataOutputStream.write(ObjectOutputStream.java:1841)
	at java.io.ObjectOutputStream.defaultWriteFields(ObjectOutputStream.java:1534)
	at java.io.ObjectOutputStream.writeSerialData(ObjectOutputStream.java:1509)
	at java.io.ObjectOutputStream.writeOrdinaryObject(ObjectOutputStream.java:1432)
	at java.io.ObjectOutputStream.writeObject0(ObjectOutputStream.java:1178)
	at java.io.ObjectOutputStream.defaultWriteFields(ObjectOutputStream.java:1548)
	at java.io.ObjectOutputStream.writeSerialData(ObjectOutputStream.java:1509)
	at java.io.ObjectOutputStream.writeOrdinaryObject(ObjectOutputStream.java:1432)
	at java.io.ObjectOutputStream.writeObject0(ObjectOutputStream.java:1178)
	at java.io.ObjectOutputStream.defaultWriteFields(ObjectOutputStream.java:1548)
	at java.io.Ob

## Triangle Count

In [53]:
result = g.triangleCount()
(result.sort("count", ascending=False)
 .filter('count > 0')
 .show())

+-----+------------+-------+
|count|          id|   name|
+-----+------------+-------+
|  280|         NED|    Ned|
|  185|      ROBERT| Robert|
|  181|      CERSEI| Cersei|
|  180|     CATELYN|Catelyn|
|  175|      TYRION| Tyrion|
|  152|     JOFFREY|Joffrey|
|  142|        ROBB|   Robb|
|  139|LITTLEFINGER|  Petyr|
|  137|       SANSA|  Sansa|
|  136|        ARYA|   Arya|
|  121|       JAIME|  Jaime|
|  117|         JON|    Jon|
|  102|       VARYS|  Varys|
|   97|       HOUND| Sandor|
|   82|        BRAN|   Bran|
|   75|       YOREN|  Yoren|
|   73|     PYCELLE|Pycelle|
|   68|       RENLY|  Renly|
|   62|       THEON|  Theon|
|   61|       TYWIN|  Tywin|
+-----+------------+-------+
only showing top 20 rows



## Label Propagation

In [57]:
result = g.labelPropagation(maxIter=10)
(result
 .sort("label")
 .groupby("label")
 .agg(F.collect_list("id").alias("people"))
 .sort(F.size("people"), ascending=False)
 .show(truncate=False))

+-------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|label        |people                                                           